In [ ]:
import os
import getpass

openai_api_key = getpass.getpass('Enter your OpenAI API key: ')
os.environ['OPENAI_API_KEY'] = openai_api_key

Enter your OpenAI API key: ··········


In [ ]:
!pip install python-dotenv
!pip install langchain-openai
!pip install --upgrade langchain
!pip install langchain-community
!pip install docarray

In [ ]:
! pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

208

# Embeddings

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
import numpy as np

In [ ]:
np.dot(embedding1, embedding2)

np.float64(0.9630313341962432)

In [ ]:
np.dot(embedding1, embedding3)

np.float64(0.7701484645728846)

In [ ]:
np.dot(embedding2, embedding3)

np.float64(0.7589254441077334)

# Vectorstores

In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 60.9 MB/s eta 0:0

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

208


## Similarity Search

In [ ]:
question = "is there an email i can ask for help"

In [ ]:
docs = vectordb.similarity_search(question,k=3)

In [ ]:
len(docs)

3

In [ ]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

Let's save this so we can use it later!

In [ ]:
vectordb.persist()

/tmp/ipython-input-3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## Failure modes

This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up.

Here are some edge cases that can arise - we'll fix them in the next class.

In [ ]:
question = "what did they say about matlab?"

In [ ]:
docs = vectordb.similarity_search(question,k=5)

Notice that we're getting duplicate chunks (because of the duplicate `MachineLearning-Lecture01.pdf` in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

`docs[0]` and `docs[1]` are indentical.

In [ ]:
docs[0]

Document(metadata={'creator': 'PScript5.dll Version 5.2.2', 'page_label': '9', 'creationdate': '2008-07-11T11:25:23-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page': 8, 'author': '', 'total_pages': 22, 'moddate': '2008-07-11T11:25:23-07:00', 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'title': ''}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t h

In [ ]:
docs[1]

Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'moddate': '2008-07-11T11:25:23-07:00', 'page': 8, 'page_label': '9', 'creationdate': '2008-07-11T11:25:23-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'title': '', 'author': '', 'total_pages': 22, 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t h

We can see a new failure mode.

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [ ]:
question = "what did they say about regression in the third lecture?"

In [ ]:
docs = vectordb.similarity_search(question,k=5)

In [ ]:
for doc in docs:
    print(doc.metadata)

{'author': '', 'creationdate': '2008-07-11T11:25:03-07:00', 'page': 0, 'title': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 16, 'moddate': '2008-07-11T11:25:03-07:00', 'page_label': '1'}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture02.pdf', 'creationdate': '2008-07-11T11:25:05-07:00', 'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'page': 2, 'total_pages': 18, 'page_label': '3', 'moddate': '2008-07-11T11:25:05-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'title': ''}
{'title': '', 'page': 17, 'creationdate': '2008-07-11T11:25:05-07:00', 'moddate': '2008-07-11T11:25:05-07:00', 'page_label': '18', 'source': 'docs/cs229_lectures/MachineLearning-Lecture02.pdf', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 18, 'author': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)'}
{'moddate': '2008-07-11T11:25:23-07:00', 'creator': '

In [ ]:
print(docs[4].page_content)

into his office and he said, "Oh, professor, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's helped me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "Wow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data networks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutorial in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical thing is the discussion sections. So discussion 
sections will be taught by the TAs, and attendance at discussion sections is o